In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**This is another classification task. we have to predict the stroke cases from the given data points (both health and personal life related)
This notebook primarily shows basic EDA and its interpretation
Dataset is quite unbalanced thus models feels overfitted.**

****

In [ ]:
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
data=pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data.info()

It is very clear that id will not contribution anything to analysis and prediction.

In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
data

We know that there are some null values in BMI. So lets fix that first

I dont why but before doing that I want to find correlation matrix

In [ ]:
#Correlation:  Its the most basic way to find degree of relation between any two quantities.
corr = data.corr()

plt.figure(figsize=(15,15))

sns.heatmap(corr,vmax=0.9,square=True)
plt.show()

Kind of surprise that BMI has very less correlation with stroke and heart disease

Lets fill the null values in BMI again and see what happens

In [ ]:
data["bmi"].fillna(data["bmi"].median(), inplace = True)

In [ ]:
data.info()

In [ ]:
#Correlation:  Its the most basic way to find degree of relation between any two quantities.
corr = data.corr()

plt.figure(figsize=(15,15))

sns.heatmap(corr,vmax=0.9,square=True)
plt.show()

Its very tempting to remove bmi but lets wait 

In [ ]:
sns.pairplot(data)

One basic conclusion which we can draw here is that young people are have less chances of getting stroke, have healthier bmi,less avg glucose level, fewer cases of hearth diseases and less hypertension too (kinda obvious).

But but but..... still not able to digest less correlation of bmi.

In [ ]:
data.corr()["stroke"]

In [ ]:
data[data['stroke']==1]['stroke'].count

We have 249 people in our data who had stroke

In [ ]:
sns.pointplot(x='stroke', y= 'age', data=data)



As we said, young people have less chances of having stroke

In [ ]:
sns.pointplot(x='stroke', y= 'avg_glucose_level', data=data)



Low glucose levels implies less risk of stroke

In [ ]:
sns.pointplot(x='stroke', y= 'bmi', data=data)

Similarly low bmi implies less chances of stroke too

Lets do some preprocessing

In [ ]:
encoder=LabelEncoder()
data["Residence_type"]=encoder.fit_transform(data["Residence_type"])
data["smoking_status"]=encoder.fit_transform(data["smoking_status"])
data["work_type"]=encoder.fit_transform(data["work_type"])
data["gender"]=encoder.fit_transform(data["gender"])
data["ever_married"]=encoder.fit_transform(data["ever_married"])




In [ ]:
scaler=StandardScaler()
data['age']=scaler.fit_transform(np.array(data['age']).reshape(-1,1))
data['avg_glucose_level']=scaler.fit_transform(np.array(data['avg_glucose_level']).reshape(-1,1))
data['bmi']=scaler.fit_transform(np.array(data['bmi']).reshape(-1,1))

In [ ]:
data

lets find correlation again

In [ ]:
data.corr()["stroke"]

OK!. So now Smoking have less correlation with stroke too. Nice. Married status matters though. This dataset really indicates to be young, smoke, remain bachelor, and its okay to have a unhealthy bmi too. 

Lets do some prediction now

In [ ]:
from sklearn.model_selection import train_test_split                
X=data.drop('stroke',axis=1,inplace=False)
y=data['stroke']
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=.2)


In [ ]:
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()
model_logistic.fit(X_train,Y_train)

accuracy=accuracy_score(Y_test,model_logistic.predict(X_test))
print(accuracy)

Good results with basic logistic too.
Lets drop the bmi column now and see

In [ ]:
X_=data.drop(['stroke','bmi','smoking_status'],axis=1,inplace=False)
y_=data['stroke']
X_train_,X_test_,Y_train_,Y_test_ = train_test_split(X_,y_,test_size=.2)
X_train_

In [ ]:
model_logistic_ = LogisticRegression()
model_logistic_.fit(X_train_,Y_train_)

accuracy_=accuracy_score(Y_test,model_logistic_.predict(X_test_))
print(accuracy)

Well, as we can see bmi and smoking_status doesnt affects model performance.
Eating whatever you like and smoking may keep your brain ok i guess.

Lets see what neural network predicts

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(8,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(X, y,validation_split=0.33, epochs=20, batch_size=5)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train', 'test'], loc='upper left')

Well it seems neural network is performing too well.
Lets introduce dropout and reduce the number of epochs

In [ ]:
model_ = Sequential()
model_.add(Dense(32, input_dim=10, activation='relu',kernel_initializer='glorot_uniform',))
model_.add(Dropout(0.2))
model_.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
model_.add(Dropout(0.2))
model_.add(Dense(8,activation='relu',kernel_initializer='glorot_uniform'))
model_.add(Dropout(0.2))
model_.add(Dense(1, activation='sigmoid'))
# Compile model
model_.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history_=model_.fit(X, y,validation_split=0.33, epochs=10, batch_size=5)

In [ ]:
plt.plot(history_.history['accuracy'])
plt.plot(history_.history['val_accuracy'])
plt.legend(['train', 'test'], loc='upper left')

Cant help, data is too simple for a neural network.

In [ ]:
data[data['stroke']==1]['stroke'].count

Well the data seems quite unbalanced too. That could be one of the reason for such behaviour